## Exercise 1
#### (c) LOOCV

## Exercise 2
#### (c) high bias and low variance

## Exercise 3
#### (b) low bias and high variance

## Exercise 4
#### Explain what regularization is and why it is useful.

Regularization, also known as shrinkage, is a technique that involves fitting a model involving all the input variables. This technique is known for tuning the function by adding an additional penalty term in the error function. This penalty is useful because the estimated coefficients are shrinken towards zero, which causes an effect of reducing the variance.

## Exercise 5

In [181]:
# (a)
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabrielferreira-data-455-bucket'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "framingham.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head(1)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0


In [182]:
## Dropping NAs
heart = heart.dropna()
heart = heart.reset_index(drop = True)

In [192]:
# Defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

#fitting data
scaler = MinMaxScaler().fit(X)
X = pd.DataFrame(scaler.transform(X), columns = X.columns)

# Defining list to store results
md1_f1 = []
md2_f1 = []

kfold = KFold(n_splits = 5, shuffle = True)

for train_ix, test_ix in kfold.split(X):
    
    ## Splitting data into train and validation
    X_train, X_val = X.loc[train_ix], X.loc[test_ix]
   
    # Defining input and target variables for the new model
    X_train_new = X_train.drop(columns = ['sysBP', 'diaBP'], axis = 1)
    X_val_new = X_val.drop(columns = ['sysBP', 'diaBP'], axis = 1)
    
    Y_train, Y_val = Y.loc[train_ix], Y.loc[test_ix]
    
   
    # Building a logistic regression model
    logit_md1 = LogisticRegression().fit(X_train, Y_train)
    logit_md2 = LogisticRegression().fit(X_train_new, Y_train)

    # Getting predictions from logit_md1 on X_test dataset
    md1_preds = logit_md1.predict_proba(X_val)[:,1]
    md2_preds = logit_md2.predict_proba(X_val_new)[:,1]
    
    # Defining likelyhoods to labels
    md1_preds = np.where(md1_preds < 0.25, 0, 1)
    md2_preds = np.where(md2_preds < 0.25, 0, 1)
    
    # Computing the F1-Score for model 1
    md1_f1.append(f1_score(Y_val, md1_preds))
    md2_f1.append(f1_score(Y_val, md2_preds))
 
    # Calculating the average of the F1-Scores
    md1_avg_f1 = np.mean(md1_f1)
    md2_avg_f1 = np.mean(md2_f1)
    
    ####
    
    # Building a logistic regression model
    logit_md2 = LogisticRegression().fit(X_train_new, Y_train)
    
    # Getting predictions from logit_md1 on X_test dataset
    md2_preds = logit_md2.predict_proba(X_val_new)[:,1]
    
    # Defining likelyhoods to labels
    md2_preds = np.where(md2_preds < 0.25, 0, 1)
    
    # Computing the F1-Score for model 2
    md2_f1.append(f1_score(Y_val, md2_preds))
    
    # Calculating the average of the F1-Scores
    md2_avg_f1 = np.mean(md2_f1)

In [193]:
print("The average F1-score of the model 1 is ", round(md1_avg_f1*100, 2), "%")
print("The average F1-score of the model 2 is ", round(md2_avg_f1*100, 2), "%")

The average F1-score of the model 1 is  34.09 %
The average F1-score of the model 2 is  31.41 %


#### I would use the first model to predict TenYearCHD because its F1-score is higher.